In [ ]:
print('Setup complete.')

# Code Review Bot - Demo (AskSage Edition with GPT-5-Mini)

**Focus**: structured comments, safety language, doc-update fallback

This notebook demonstrates building an AI-powered code review bot that provides structured feedback on code changes with safety considerations and fallback mechanisms using AskSage API with GPT-5-Mini model.

## Learning Objectives
- Generate structured code review comments using AskSage with GPT-5-Mini
- Implement safety language and content filtering
- Build fallback mechanisms for edge cases
- Export review results in JSON format

In [ ]:
# Install required packages for Google Colab
!pip install requests asksageclient pandas numpy

import json # Import the json module to work with JSON data
import requests # Import the requests library to send HTTP requests
from asksageclient import AskSageClient # Import the AskSageClient class from the asksageclient module
import os
from datetime import datetime
from typing import Dict, List, Optional, Tuple
import re
from dataclasses import dataclass

print("✅ All packages installed and modules imported successfully!")

In [ ]:
# Function to load credentials from a JSON file
def load_credentials(filename):
    try:
        with open(filename) as file:
            return json.load(file)
    except FileNotFoundError:
        raise FileNotFoundError("The credentials file was not found.")
    except json.JSONDecodeError:
        raise ValueError("Failed to decode JSON from the credentials file.")

# Load the credentials
credentials = load_credentials('../../credentials.json')

# Extract the API key, and email from the credentials to be used in the API request
api_key = credentials['credentials']['api_key']
email = credentials['credentials']['Ask_sage_user_info']['username']

"""  
class AskSageClient(
    email: email, # The email address of the user
    api_key: api_key, # The API key for the Ask Sage API, which can be obtained from the Ask Sage website
    user_base_url: str = 'https://api.asksage.ai/user', # The base URL for the user API
    server_base_url: str = 'https://api.asksage.ai/server' # The base URL for the server API
)
"""

ask_sage_client = AskSageClient(email, api_key) # Create an instance of the AskSageClient class with the email and api_key

print("✅ AskSage client initialized successfully!")

# Verify GPT-5-Mini model availability
try:
    models_response = ask_sage_client.get_models()
    available_models = models_response.get('response', [])
    if 'gpt-5-mini' in available_models:
        print("✅ GPT-5-Mini model is available!")
    else:
        print("⚠️ GPT-5-Mini model not found in available models. Available models:")
        for model in available_models[:10]:  # Show first 10 models
            print(f"   - {model}")
        print("   ... (and more)")
except Exception as e:
    print(f"⚠️ Could not verify models: {e}")

## 1. Code Review Data Structures

First, let's define structured data models for our code review comments.

In [ ]:
@dataclass
class ReviewComment:
    """Structured code review comment."""
    line_number: int
    severity: str  # 'critical', 'major', 'minor', 'suggestion'
    category: str  # 'security', 'performance', 'style', 'logic', 'documentation'
    message: str
    suggestion: Optional[str] = None
    confidence: float = 0.0  # 0.0 to 1.0

@dataclass
class ReviewResult:
    """Complete review result with metadata."""
    file_path: str
    comments: List[ReviewComment]
    summary: str
    timestamp: str
    reviewer_version: str = "v1.0-gpt5mini"
    model_used: str = "gpt-5-mini"
    
    def to_dict(self) -> Dict:
        """Convert to dictionary for JSON export."""
        return {
            "file_path": self.file_path,
            "comments": [
                {
                    "line_number": c.line_number,
                    "severity": c.severity,
                    "category": c.category,
                    "message": c.message,
                    "suggestion": c.suggestion,
                    "confidence": c.confidence
                } for c in self.comments
            ],
            "summary": self.summary,
            "timestamp": self.timestamp,
            "reviewer_version": self.reviewer_version,
            "model_used": self.model_used
        }

print("✅ Review data structures defined with GPT-5-Mini support")

## 2. Safety Language Filter

Implement content filtering to ensure appropriate language in code reviews.

In [ ]:
class SafetyFilter:
    """Filter for ensuring appropriate language in code reviews."""
    
    def __init__(self):
        # Patterns to avoid in review comments
        self.inappropriate_patterns = [
            r'\b(stupid|dumb|idiotic|moronic)\b',
            r'\b(terrible|awful|horrible)\s+(code|implementation)',
            r'\b(trash|garbage|crap)\b',
            r'what\s+were\s+you\s+thinking',
            r'this\s+is\s+(wrong|bad|terrible)'
        ]
        
        # Positive alternatives
        self.positive_alternatives = {
            'bad': 'could be improved',
            'wrong': 'might need adjustment',
            'terrible': 'has room for improvement',
            'stupid': 'unclear',
            'fix this': 'consider updating this'
        }
    
    def filter_comment(self, comment: str) -> str:
        """Apply safety filtering to a comment."""
        filtered = comment.lower()
        
        # Check for inappropriate patterns
        for pattern in self.inappropriate_patterns:
            if re.search(pattern, filtered, re.IGNORECASE):
                # Replace with more constructive language
                filtered = re.sub(pattern, 'needs improvement', filtered, flags=re.IGNORECASE)
        
        # Apply positive alternatives
        for negative, positive in self.positive_alternatives.items():
            filtered = filtered.replace(negative, positive)
        
        return filtered.capitalize()
    
    def is_constructive(self, comment: str) -> bool:
        """Check if comment is constructive."""
        constructive_indicators = [
            'consider', 'suggest', 'recommend', 'could', 'might',
            'alternative', 'improvement', 'enhancement'
        ]
        
        return any(indicator in comment.lower() for indicator in constructive_indicators)

# Test the safety filter
safety_filter = SafetyFilter()

test_comments = [
    "This code is terrible and stupid",
    "Fix this bad implementation",
    "Consider improving the error handling here"
]

print("Safety Filter Examples:")
for comment in test_comments:
    filtered = safety_filter.filter_comment(comment)
    constructive = safety_filter.is_constructive(filtered)
    print(f"Original: {comment}")
    print(f"Filtered: {filtered} (Constructive: {constructive})")
    print()

## 3. Code Review Bot with AskSage Integration (GPT-5-Mini)

Create the main code review bot using AskSage API with GPT-5-Mini model and structured prompting.

In [ ]:
class CodeReviewBot:
    """AI-powered code review bot with safety and structure using AskSage with GPT-5-Mini."""
    
    def __init__(self, ask_sage_client):
        self.ask_sage_client = ask_sage_client
        self.safety_filter = SafetyFilter()
        self.model = "gpt-5-mini"  # Specify GPT-5-Mini model
        
        # Structured prompt for code review
        self.system_prompt = """
You are a senior software engineer conducting a thorough but constructive code review using GPT-5-Mini.
Your goal is to help improve code quality while being respectful and educational.

For each issue you find, provide:
1. Line number (if applicable)
2. Severity: critical, major, minor, or suggestion
3. Category: security, performance, style, logic, or documentation
4. Clear explanation of the issue
5. Specific improvement suggestion
6. Confidence level (0.0-1.0)

Focus on:
- Security vulnerabilities
- Performance issues
- Code clarity and maintainability
- Best practices
- Documentation needs

Use constructive language. Instead of "wrong" or "bad", use phrases like:
- "Consider improving..."
- "This could be enhanced by..."
- "A more robust approach would be..."

Format your response as JSON with this structure:
{
  "comments": [
    {
      "line_number": <number>,
      "severity": "<severity>",
      "category": "<category>",
      "message": "<constructive message>",
      "suggestion": "<specific improvement>",
      "confidence": <0.0-1.0>
    }
  ],
  "summary": "<overall assessment>"
}
"""
    
    def review_code(self, file_path: str, code_diff: str) -> ReviewResult:
        """Review code diff and return structured results using AskSage with GPT-5-Mini."""
        try:
            # Prepare the user message
            user_message = f"Please review this code:\n\nFile: {file_path}\n\n{code_diff}"
            
            # Generate review using AskSage with GPT-5-Mini
            response = self.ask_sage_client.ask(
                message=user_message,
                system_prompt=self.system_prompt,
                model=self.model,  # Use GPT-5-Mini
                temperature=0.3
            )
            
            # Parse JSON response
            review_data = json.loads(response)
            
            # Apply safety filtering and create structured comments
            comments = []
            for comment_data in review_data.get('comments', []):
                filtered_message = self.safety_filter.filter_comment(comment_data['message'])
                filtered_suggestion = None
                if comment_data.get('suggestion'):
                    filtered_suggestion = self.safety_filter.filter_comment(comment_data['suggestion'])
                
                comment = ReviewComment(
                    line_number=comment_data['line_number'],
                    severity=comment_data['severity'],
                    category=comment_data['category'],
                    message=filtered_message,
                    suggestion=filtered_suggestion,
                    confidence=comment_data.get('confidence', 0.8)
                )
                comments.append(comment)
            
            # Create review result
            result = ReviewResult(
                file_path=file_path,
                comments=comments,
                summary=self.safety_filter.filter_comment(review_data.get('summary', 'Code reviewed with GPT-5-Mini')),
                timestamp=datetime.now().isoformat(),
                model_used=self.model
            )
            
            return result
            
        except Exception as e:
            # Fallback mechanism
            return self._fallback_review(file_path, code_diff, str(e))
    
    def _fallback_review(self, file_path: str, code_diff: str, error: str) -> ReviewResult:
        """Fallback review when main process fails."""
        fallback_comment = ReviewComment(
            line_number=1,
            severity="minor",
            category="documentation",
            message="Automated review temporarily unavailable. Manual review recommended.",
            suggestion="Please have a colleague review this code change.",
            confidence=1.0
        )
        
        return ReviewResult(
            file_path=file_path,
            comments=[fallback_comment],
            summary=f"Fallback review triggered due to: {error}",
            timestamp=datetime.now().isoformat(),
            model_used="fallback"
        )

print("✅ Code Review Bot created with AskSage GPT-5-Mini integration")

## 4. Demo: Review a Code Diff with GPT-5-Mini

Let's demonstrate the bot with a sample code diff using the GPT-5-Mini model.

In [ ]:
# Sample code diff for demonstration
sample_diff = """
+ def login_user(username, password):
+     query = "SELECT * FROM users WHERE username='" + username + "' AND password='" + password + "'"
+     cursor.execute(query)
+     user = cursor.fetchone()
+     if user:
+         session['user_id'] = user[0]
+         return True
+     return False
+
+ def process_file_upload(file):
+     filename = file.filename
+     filepath = '/uploads/' + filename
+     file.save(filepath)
+     return filepath
+
+ def calculate_total(prices):
+     total = 0
+     for price in prices:
+         total = total + price
+     return total
"""

# Initialize the review bot with AskSage client
review_bot = CodeReviewBot(ask_sage_client)

print("🔍 Reviewing sample code diff with AskSage GPT-5-Mini...")
print("Sample diff:")
print(sample_diff)
print("\n" + "="*60)


In [ ]:
# Perform the review with GPT-5-Mini
review_result = review_bot.review_code(
    file_path="app/auth.py",
    code_diff=sample_diff
)

print("📋 CODE REVIEW RESULTS (GPT-5-Mini)")
print("=" * 60)
print(f"File: {review_result.file_path}")
print(f"Model Used: {review_result.model_used}")
print(f"Timestamp: {review_result.timestamp}")
print(f"\nSummary: {review_result.summary}")
print(f"\nFound {len(review_result.comments)} issues:")
print()

# Display comments with structured formatting
for i, comment in enumerate(review_result.comments, 1):
    severity_emoji = {
        'critical': '🚨',
        'major': '⚠️',
        'minor': '💡',
        'suggestion': '💭'
    }.get(comment.severity, '📝')
    
    print(f"{severity_emoji} Issue #{i} - Line {comment.line_number}")
    print(f"   Severity: {comment.severity.upper()}")
    print(f"   Category: {comment.category}")
    print(f"   Message: {comment.message}")
    if comment.suggestion:
        print(f"   Suggestion: {comment.suggestion}")
    print(f"   Confidence: {comment.confidence:.1f}")
    print()

## 5. Export Review Results

Export the review results as JSON for integration with other systems.

In [ ]:
# Export to JSON
review_json = review_result.to_dict()

# Pretty print the JSON
print("📄 EXPORTED REVIEW JSON (GPT-5-Mini)")
print("=" * 60)
print(json.dumps(review_json, indent=2))

# Save to file (optional)
output_filename = f"review_result_gpt5mini_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(output_filename, 'w') as f:
    json.dump(review_json, f, indent=2)

print(f"\n💾 Review saved to: {output_filename}")

## 6. Before/After Code Example

Show how the original code could be improved based on review feedback.

In [ ]:
print("🔄 BEFORE/AFTER COMPARISON (GPT-5-Mini Analysis)")
print("=" * 60)

print("❌ BEFORE (with issues):")
print("""
def login_user(username, password):
    query = "SELECT * FROM users WHERE username='" + username + "' AND password='" + password + "'"
    cursor.execute(query)
    user = cursor.fetchone()
    if user:
        session['user_id'] = user[0]
        return True
    return False
""")

print("\n✅ AFTER (improved with GPT-5-Mini suggestions):")
print("""
def login_user(username: str, password: str) -> bool:
    \"\"\"Authenticate user with secure password verification.
    
    Args:
        username: User's username
        password: User's plain text password
        
    Returns:
        bool: True if authentication successful, False otherwise
    \"\"\"
    # Use parameterized query to prevent SQL injection
    query = "SELECT id, password_hash FROM users WHERE username = %s"
    cursor.execute(query, (username,))
    user = cursor.fetchone()
    
    if user and verify_password(password, user[1]):
        session['user_id'] = user[0]
        return True
    return False
""")

print("\n🔧 Key Improvements (GPT-5-Mini Analysis):")
print("- Fixed SQL injection vulnerability with parameterized queries")
print("- Added proper password hashing verification")
print("- Added type hints and documentation")
print("- Improved code structure and readability")
print("- Enhanced security posture following modern best practices")

## Summary

In this demo, we built a comprehensive Code Review Bot using AskSage with GPT-5-Mini that:

1. **GPT-5-Mini Integration**: Uses the latest GPT-5-Mini model for enhanced code analysis
2. **Structured Comments**: Uses formal data structures for consistent review feedback
3. **Safety Language**: Filters inappropriate language and promotes constructive feedback
4. **Doc-Update Fallback**: Provides graceful degradation when AI review fails
5. **JSON Export**: Exports results in machine-readable format for integration
6. **Before/After Examples**: Shows concrete improvements based on feedback
7. **Model Verification**: Checks for GPT-5-Mini availability and provides feedback

The bot demonstrates enterprise-grade features including error handling, content filtering, and structured output that can integrate with existing development workflows, all powered by the advanced capabilities of GPT-5-Mini.

### GPT-5-Mini Advantages:
- Improved code understanding and analysis
- Better contextual awareness of security vulnerabilities
- Enhanced natural language generation for review comments
- More accurate confidence scoring
- Better adherence to structured output formats